In [41]:
import os
import glob
import librosa
import IPython.display as ipd
from pydub import AudioSegment
from pydub.playback import play
import soundfile as sf
from torchmetrics import WordErrorRate
import string
from fnmatch import fnmatch
from tqdm import tqdm
import nemo.collections.asr as nemo_asr
import nemo.collections.nlp as nemo_nlp

In [42]:
m4a_version = AudioSegment.from_file('Website/audio/Recording'+'.m4a', "m4a")
m4a_version.export('Website/audio/Recording'+'.wav', format='wav')
audio, sample_rate = librosa.load('Website/audio/Recording'+'.wav', sr=16000)
sf.write('Website/audio/Recording'+'_resampled.wav', audio, sample_rate)

In [43]:
sample_rate

16000

In [44]:
model = nemo_asr.models.EncDecCTCModel.restore_from(restore_path=os.path.join('Website/models', "conformer_transducer.nemo"))
puncbert_model = nemo_nlp.models.PunctuationCapitalizationModel.restore_from(restore_path=os.path.join('Website/models', "punc_bert.nemo"))

[NeMo I 2023-04-07 09:59:26 mixins:170] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2023-04-07 09:59:27 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/NeMo_ASR_SET/English/v2.0/train/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 32
    shuffle: true
    num_workers: 8
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 20.0
    min_duration: 0.1
    is_tarred: true
    tarred_audio_filepaths: /data/NeMo_ASR_SET/English/v2.0/train/audio__OP_0..4095_CL_.tar
    shuffle_n: 2048
    bucketing_strategy: synced_randomized
    bucketing_batch_size: null
    
[NeMo W 2023-04-07 09:59:27 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation confi

[NeMo I 2023-04-07 09:59:27 features:286] PADDING: 0


[NeMo W 2023-04-07 09:59:28 nemo_logging:349] /usr/local/lib/python3.8/dist-packages/torch/nn/modules/rnn.py:67: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
      warnings.warn("dropout option adds dropout after all but last "
    


[NeMo I 2023-04-07 09:59:28 rnnt_models:206] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0}
[NeMo I 2023-04-07 09:59:29 save_restore_connector:247] Model EncDecRNNTBPEModel was successfully restored from /root/code/PentaHack/Website/models/conformer_transducer.nemo.
[NeMo I 2023-04-07 09:59:30 tokenizer_utils:130] Getting HuggingFace AutoTokenizer with pretrained_model_name: distilbert-base-uncased, vocab_file: /tmp/tmpii02zroi/tokenizer.vocab_file, merges_files: None, special_tokens_dict: {}, and use_fast: False


Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.
[NeMo W 2023-04-07 09:59:31 modelPT:245] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.
[NeMo W 2023-04-07 09:59:31 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    use_audio: false
    audio_file: null
    sample_rate: 16000
    use_bucketing: true
    batch_size: 32
    preload_audios: true
    use_tarred_dataset: false
    label_info_save_dir: null
    text_file: text_train.txt
    labels_file: labels_train.txt
    tokens_in_batch: null
    max_seq_length: 128
    num_samples: -1
    use_cache: true
    cache_dir: null
    get_label_frequences: false
    verbose: true
    n_jobs: 0
    tar_metadata_file: null
    tar_shuffle_n: 1
    shard_strategy: scatter
    shuffle: true

[NeMo I 2023-04-07 09:59:36 save_restore_connector:247] Model PunctuationCapitalizationModel was successfully restored from /root/code/PentaHack/Website/models/punc_bert.nemo.


In [45]:
transcribed_text = model.transcribe(paths2audio_files=[os.path.join('Website/audio', "Recording_resampled.wav")])

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [52]:
transformed_text = puncbert_model.add_punctuation_capitalization(transcribed_text[0])
remove_last_index = False
if (transformed_text[0][-1] == '.') | (transformed_text[0][-1] == '!'):
    remove_last_index = True

[NeMo I 2023-04-07 10:09:45 punctuation_capitalization_model:1153] Using batch size 1 for inference
[NeMo I 2023-04-07 10:09:45 punctuation_capitalization_infer_dataset:127] Max length: 42
[NeMo I 2023-04-07 10:09:45 data_preprocessing:404] Some stats of the lengths of the sequences:
[NeMo I 2023-04-07 10:09:45 data_preprocessing:406] Min: 40 |                  Max: 40 |                  Mean: 40.0 |                  Median: 40.0
[NeMo I 2023-04-07 10:09:45 data_preprocessing:412] 75 percentile: 40.00
[NeMo I 2023-04-07 10:09:45 data_preprocessing:413] 99 percentile: 40.00


100%|██████████| 1/1 [00:00<00:00, 10.60batch/s]


In [53]:
transformed_text

['The bell went off signalling the end of the school day after saying good-bye to his classmates. Jack walked hurriedly to the bus stop, hoping to return home as soon as possible to watch his favourite television show.']

In [54]:
from transformers import pipeline

classifier = pipeline("text-classification", model="bhadresh-savani/bert-base-go-emotion")

In [55]:
paragraph_split = transformed_text[0].strip().split('.')

In [56]:
paragraph_split

['The bell went off signalling the end of the school day after saying good-bye to his classmates',
 ' Jack walked hurriedly to the bus stop, hoping to return home as soon as possible to watch his favourite television show',
 '']

In [57]:
sentence_dict = {}
if remove_last_index:
    for index, sentence in enumerate(paragraph_split):
        print(index)
        if index < len(paragraph_split)-1:
            sentence_dict[sentence] = classifier(sentence)[0]['label']

0
1
2


In [15]:
sentence_dict

{'The bell went off signalling the end of the school day after saying good-bye to his classmates': 'neutral',
 ' Jack walked hurriedly to the bus stop, hoping to return home as soon as possible to watch his favourite television show': 'optimism'}

In [16]:
new_text =f''

for sentence in sentence_dict:
    new_text+=(f'{sentence}[{sentence_dict[sentence]}]')
print(new_text)

The bell went off signalling the end of the school day after saying good-bye to his classmates[neutral] Jack walked hurriedly to the bus stop, hoping to return home as soon as possible to watch his favourite television show[optimism]


In [39]:
# Load FastPitch
from nemo.collections.tts.models import FastPitchModel
spec_generator = FastPitchModel.restore_from(restore_path=os.path.join('Website/models', "fastpitch.nemo"))

# Load vocoder
from nemo.collections.tts.models import HifiGanModel
model = HifiGanModel.restore_from(restore_path=os.path.join('Website/models', "hifigan.nemo"))


[NeMo I 2023-04-07 09:02:09 tokenize_and_classify:87] Creating ClassifyFst grammars.


[NeMo W 2023-04-07 09:02:41 modules:129] apply_to_oov_word=None, This means that some of words will remain unchanged if they are not handled by any of the rules in self.parse_one_word(). This may be intended if phonemes and chars are both valid inputs, otherwise, you may see unexpected deletions in your input.
[NeMo W 2023-04-07 09:02:41 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.torch.data.TTSDataset
      manifest_filepath: /ws/LJSpeech/nvidia_ljspeech_train_clean_ngc.json
      sample_rate: 22050
      sup_data_path: /raid/LJSpeech/supplementary
      sup_data_types:
      - align_prior_matrix
      - pitch
      n_fft: 1024
      win_length: 1024
      hop_length: 256
      window: hann
      n_mels: 80
      lowfreq: 0
      highfreq: 8000
      max_duration: null
      min_d

[NeMo I 2023-04-07 09:02:41 features:286] PADDING: 1
[NeMo I 2023-04-07 09:02:42 save_restore_connector:247] Model FastPitchModel was successfully restored from /root/code/PentaHack/Website/models/fastpitch.nemo.


[NeMo W 2023-04-07 09:02:42 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/train_finetune.txt
      min_duration: 0.75
      n_segments: 8192
    dataloader_params:
      drop_last: false
      shuffle: true
      batch_size: 64
      num_workers: 4
    
[NeMo W 2023-04-07 09:02:42 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    dataset:
      _target_: nemo.collections.tts.data.datalayers.MelAudioDataset
      manifest_filepath: /home/fkreuk/data/val_finetune.txt
      min_duration: 3
      n_segments: 66150


[NeMo I 2023-04-07 09:02:42 features:286] PADDING: 0


[NeMo W 2023-04-07 09:02:42 features:263] Using torch_stft is deprecated and has been removed. The values have been forcibly set to False for FilterbankFeatures and AudioToMelSpectrogramPreprocessor. Please set exact_pad to True as needed.


[NeMo I 2023-04-07 09:02:42 features:286] PADDING: 0
[NeMo I 2023-04-07 09:02:44 save_restore_connector:247] Model HifiGanModel was successfully restored from /root/code/PentaHack/Website/models/hifigan.nemo.


In [37]:
import soundfile as sf
userinput = "You can type your sentence here to get nemo to produce speech."
parsed = spec_generator.parse(userinput)
spectrogram = spec_generator.generate_spectrogram(tokens=parsed)
audio = model.convert_spectrogram_to_audio(spec=spectrogram)


[NeMo W 2023-04-07 08:56:42 fastpitch:262] parse() is meant to be called in eval mode.
[NeMo W 2023-04-07 08:56:42 fastpitch:327] generate_spectrogram() is meant to be called in eval mode.


In [38]:
# Save the audio to disk in a file called speech.wav
sf.write("speech.wav", audio.to('cpu').detach().numpy()[0], 22050)
